In [15]:
import argparse
import re
import sys
import zipfile
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



In [16]:

def ensure_dir(path: Path):
    path.mkdir(parents=True, exist_ok=True)


def unzip_if_needed(path: Path, work_root: Path) -> Path:
    if path.is_dir():
        return path
    if zipfile.is_zipfile(path):
        dest = work_root / path.stem
        if dest.exists():
            return dest
        ensure_dir(dest)
        with zipfile.ZipFile(path, "r") as zf:
            zf.extractall(dest)
        return dest
    raise FileNotFoundError(f"Not a directory or zip: {path}")


def debug_inventory(root: Path, label: str):
    print(f"\n[debug] Scanning {label} at {root}")
    csvs = list(root.rglob("*.csv"))
    if not csvs:
        print("[debug]   No CSV files found.")
        return
    for f in sorted(csvs)[:200]:
        try:
            df = pd.read_csv(f, nrows=2)
            cols = ", ".join(map(str, df.columns.tolist()))
        except Exception as e:
            cols = f"<could not read: {e}>"
        try:
            rel = f.relative_to(root)
        except Exception:
            rel = f
        print(f"[debug]   {rel} -> cols: {cols}")


# ---------------------------- Loaders ----------------------------

def load_runtime_from_adp_runtime_by_N(root: Path) -> Dict[str, np.ndarray]:
    cand = list(root.rglob("adp_runtime_by_N.csv"))
    if not cand:
        return {"N": np.array([]), "runtime_h": np.array([])}
    df = pd.read_csv(cand[0])
    if "N" in df.columns and "runtime_hours" in df.columns:
        Ns = pd.to_numeric(df["N"], errors="coerce").dropna().astype(int).to_numpy()
        H  = pd.to_numeric(df["runtime_hours"], errors="coerce").dropna().to_numpy()
        idx = np.argsort(Ns)
        return {"N": Ns[idx], "runtime_h": H[idx]}
    return {"N": np.array([]), "runtime_h": np.array([])}


def load_runtime_from_run_summary(root: Path) -> Dict[str, np.ndarray]:
    cand = list(root.rglob("run_summary_allN.csv"))
    if not cand:
        return {"N": np.array([]), "runtime_h": np.array([])}
    df = pd.read_csv(cand[0])
    if "N" in df.columns and "runtime_hours" in df.columns:
        Ns = pd.to_numeric(df["N"], errors="coerce").dropna().astype(int).to_numpy()
        H  = pd.to_numeric(df["runtime_hours"], errors="coerce").dropna().to_numpy()
        idx = np.argsort(Ns)
        return {"N": Ns[idx], "runtime_h": H[idx]}
    return {"N": np.array([]), "runtime_h": np.array([])}


def load_std_from_perN_files(root: Path) -> Dict[int, pd.DataFrame]:
    by_N: Dict[int, pd.DataFrame] = {}
    for pth in root.rglob("adp_N*_std_last5_by_time.csv"):
        m = re.search(r"adp_N(\d+)_std_last5_by_time\.csv$", pth.name)
        if not m:
            continue
        N = int(m.group(1))
        df = pd.read_csv(pth)
        row = df.iloc[0]
        hour_cols = [c for c in df.columns if isinstance(c, str) and c.lower().startswith("hour")]
        if not hour_cols:
            hour_cols = df.columns.tolist()
        data = []
        for c in hour_cols:
            m2 = re.search(r'(\d+)$', str(c))
            t = int(m2.group(1)) if m2 else hour_cols.index(c) + 1
            v = pd.to_numeric(row[c], errors="coerce")
            if pd.isna(v):
                continue
            data.append((t, float(v)))
        if data:
            data.sort(key=lambda x: x[0])
            by_N[N] = pd.DataFrame(data, columns=["time", "value"]).assign(sample_id=0)
    return by_N


def load_eev_ws_from_pair_files(root: Path) -> Optional[pd.DataFrame]:
    rows = []
    for p in root.rglob("in_sample_N*.csv"):
        m = re.search(r"in_sample_N(\d+)\.csv$", p.name)
        if not m:
            continue
        N = int(m.group(1))
        df_in = pd.read_csv(p)
        q = p.with_name(f"out_of_sample_N{N}.csv")
        if not q.exists():
            continue
        df_out = pd.read_csv(q)
        for approach in ["EEV", "WS", "ADP", "SDDP"]:
            if approach in df_in.columns and approach in df_out.columns:
                rows.append({
                    "approach": approach,
                    "N": N,
                    "in_sample": float(pd.to_numeric(df_in[approach], errors="coerce").mean()),
                    "out_of_sample": float(pd.to_numeric(df_out[approach], errors="coerce").mean()),
                })
    if not rows:
        return None
    return pd.DataFrame(rows).sort_values(["approach", "N"])


# --- replace the whole function ---
def load_first_hour_auto(root: Path) -> Optional[Tuple[np.ndarray, np.ndarray, str]]:
    """
    Return (x, y, xname). If no level/state column exists, fall back to 'path'
    so we at least plot value-by-scenario index (not a true value function).
    """
    candidates = list(root.rglob("run_N*_first_hour.csv"))
    if not candidates:
        return None

    def N_of(path):
        m = re.search(r"run_N(\d+)_first_hour\.csv$", path.name, re.IGNORECASE)
        return int(m.group(1)) if m else -1

    candidates.sort(key=lambda q: N_of(q), reverse=True)

    for csv_path in candidates:
        try:
            df = pd.read_csv(csv_path)
        except Exception:
            continue

        # y (value)
        ycol = None
        for cand in ["value","vbar2_at_xpost1","Vhat_hour1","v_est","vhat","v"]:
            if cand in df.columns:
                ycol = cand; break
        if ycol is None:
            nums = df.select_dtypes(include=["number"]).columns.tolist()
            if not nums: 
                continue
            ycol = nums[-1]

        # x (level/state) preferred list
        xcol = None
        preferred = ["level","post_level","xpost1","x_post","xpost","Reservoir1","Reservoir 1","state","l","storage"]
        for cand in preferred:
            if cand in df.columns:
                xcol = cand; break

        # Fallbacks:
        #   1) 'path' if present (scenario index)
        #   2) any numeric column ≠ y (even if it's price/inflow/theta)
        if xcol is None:
            if "path" in df.columns:
                xcol = "path"
            else:
                nums = [c for c in df.select_dtypes(include=["number"]).columns if c != ycol]
                if not nums:
                    continue
                xcol = max(nums, key=lambda c: pd.to_numeric(df[c], errors="coerce").var())

        x = pd.to_numeric(df[xcol], errors="coerce").dropna().to_numpy()
        y = pd.to_numeric(df[ycol], errors="coerce").dropna().to_numpy()
        if len(x) and len(y):
            return (x, y, xcol)

    return None






# ---------------------------- New helpers for 4/5/6/7 ----------------------------

def write_eev_ws_table_tex(eevws_df: pd.DataFrame, out_tex: Path):
    df = eevws_df.copy()
    Ns = sorted(df["N"].unique())
    lines = []
    lines.append(r"\begin{table}[ht]")
    lines.append(r"\centering")
    lines.append(r"\caption{Mean profit across in-sample and out-of-sample scenarios.}")
    lines.append(r"\begin{tabular}{l" + "c" * (2 * len(Ns)) + "}")
    lines.append(r"\toprule")
    header = ["Approach"] + [f"In-sample ($N={n}$)" for n in Ns] + [f"Out-of-sample ($N={n}$)" for n in Ns]
    lines.append(" & ".join(header) + r" \\")
    lines.append(r"\midrule")
    for approach in ["EEV", "WS", "ADP", "SDDP"]:
        if approach not in df["approach"].unique():
            continue
        row_in, row_out = [], []
        for n in Ns:
            sub = df[(df["approach"] == approach) & (df["N"] == n)]
            if sub.empty:
                row_in.append("")
                row_out.append("")
            else:
                row_in.append(f"{pd.to_numeric(sub['in_sample']).values[0]:.4f}")
                row_out.append(f"{pd.to_numeric(sub['out_of_sample']).values[0]:.4f}")
        lines.append(" & ".join([approach] + row_in + row_out) + r" \\")
    lines.append(r"\bottomrule")
    lines.append(r"\end{tabular}")
    lines.append(r"\label{tab:eev_ws}")
    lines.append(r"\end{table}")
    out_tex.write_text("\n".join(lines), encoding="utf-8")


def load_oos_stack(eev_root: Path) -> Optional[pd.DataFrame]:
    rows = []
    for pth in eev_root.rglob("out_of_sample_N*.csv"):
        m = re.search(r"out_of_sample_N(\d+)\.csv$", pth.name)
        if not m:
            continue
        N = int(m.group(1))
        df = pd.read_csv(pth)
        for col in df.columns:
            if col.strip().lower() in {"scenario"}:
                continue
            series = pd.to_numeric(df[col], errors="coerce").dropna()
            for v in series.values:
                rows.append({"N": N, "approach": col, "value": float(v)})
    if not rows:
        return None
    return pd.DataFrame(rows)


def plot_oos_distributions_by_N(oos_long: pd.DataFrame, outdir: Path):
    for N, grp in oos_long.groupby("N"):
        plt.figure(figsize=(6.0, 4.0), dpi=150)
        approaches = sorted(grp["approach"].unique())
        data = [grp[grp["approach"] == a]["value"].to_numpy() for a in approaches]
        plt.boxplot(data, labels=approaches, showfliers=False)
        plt.xlabel("Approach")
        plt.ylabel("Out-of-sample profit ($)")
        plt.title(f"Out-of-sample distributions (N={N})")
        plt.tight_layout()
        plt.savefig(outdir / f"fig_oos_dist_N{N}.pdf", bbox_inches="tight")
        plt.close()


def load_runtime_generic(root: Path) -> Dict[str, np.ndarray]:
    rt = load_runtime_from_adp_runtime_by_N(root)
    if not rt["N"].size:
        rt = load_runtime_from_run_summary(root)
    return rt


def compute_efficiency_points(runtime_root: Path, oos_long: Optional[pd.DataFrame], approaches: List[str]) -> Optional[pd.DataFrame]:
    if oos_long is None:
        return None
    rows = []
    rt = load_runtime_generic(runtime_root)
    if not rt["N"].size:
        return None
    rt_map = {int(n): float(h) for n, h in zip(rt["N"], rt["runtime_h"])}
    for approach in approaches:
        sub = oos_long[oos_long["approach"].str.lower() == approach.lower()]
        if sub.empty:
            continue
        for N, grp in sub.groupby("N"):
            quality = float(pd.to_numeric(grp["value"], errors="coerce").mean())
            rh = rt_map.get(int(N))
            if rh is None:
                continue
            rows.append({"approach": approach.upper(), "N": int(N), "runtime_h": rh, "quality": quality})
    if not rows:
        return None
    return pd.DataFrame(rows).sort_values(["approach","N"])


def plot_efficiency_frontier(points_df: pd.DataFrame, out: Path):
    plt.figure(figsize=(6.0, 4.0), dpi=150)
    for appr, grp in points_df.groupby("approach"):
        grp = grp.sort_values("runtime_h")
        plt.plot(grp["runtime_h"], grp["quality"], marker="o", label=appr)
        for _, r in grp.iterrows():
            plt.annotate(str(int(r["N"])), (r["runtime_h"], r["quality"]), xytext=(3,3), textcoords="offset points")
    plt.xlabel("Runtime (h)")
    plt.ylabel("Out-of-sample mean profit ($)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out, bbox_inches="tight")
    plt.close()


def load_policy_t1_pair(root: Path) -> Optional[Tuple[np.ndarray, np.ndarray]]:
    # levels
    lev = None
    for pth in root.rglob("*lastpath*_levels.csv"):
        lev = pth; break
    if lev is None:
        return None
    try:
        dL = pd.read_csv(lev)
    except Exception:
        return None

    # time column (or fall back to first row)
    tcol = next((c for c in dL.columns if c.strip().lower() in {"time","hour","t","stage"}), None)
    if tcol is not None:
        Lrow = dL[dL[tcol] == dL[tcol].min()]
    else:
        Lrow = dL.head(1)

    level_cols = [c for c in Lrow.select_dtypes(include=["number"]).columns if c != tcol]
    if not level_cols:
        return None
    level_t1 = Lrow[level_cols[0]].to_numpy()

    # decisions
    pi = None
    for pth in root.rglob("*lastpath*_pi.csv"):
        pi = pth; break
    if pi is None:
        return None
    try:
        dP = pd.read_csv(pi)
    except Exception:
        return None

    tcolP = next((c for c in dP.columns if c.strip().lower() in {"time","hour","t","stage"}), None)
    if tcolP is not None:
        Prow = dP[dP[tcolP] == dP[tcolP].min()]
    else:
        Prow = dP.head(1)

    dec_cols = [c for c in Prow.select_dtypes(include=["number"]).columns if c != tcolP]
    if not dec_cols:
        return None
    # pick the most varying numeric as decision if multiple
    dec_col = max(dec_cols, key=lambda c: pd.to_numeric(dP[c], errors="coerce").var())
    pi_t1 = Prow[dec_col].to_numpy()

    if level_t1.size and pi_t1.size:
        return (level_t1, pi_t1)
    return None



def plot_policy_t1(adp_pair: Optional[Tuple[np.ndarray, np.ndarray]], sddp_pair: Optional[Tuple[np.ndarray, np.ndarray]], out: Path):
    if adp_pair is None and sddp_pair is None:
        return
    plt.figure(figsize=(6.0, 4.0), dpi=150)
    if adp_pair is not None and adp_pair[0].size and adp_pair[1].size:
        plt.plot(adp_pair[0], adp_pair[1], marker="o", linestyle="None", label="ADP")
    if sddp_pair is not None and sddp_pair[0].size and sddp_pair[1].size:
        plt.plot(sddp_pair[0], sddp_pair[1], marker="+", linestyle="None", label="SDDP")
    plt.xlabel("Level at t=1")
    plt.ylabel("Dispatch / decision at t=1")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out, bbox_inches="tight")
    plt.close()


# ---------------------------- Plotters ----------------------------

def plot_runtime(adp_rt: Dict[str, np.ndarray], sddp_rt: Dict[str, np.ndarray], out: Path):
    plt.figure(figsize=(6.0, 4.0), dpi=150)
    if adp_rt["N"].size:
        plt.plot(adp_rt["N"], adp_rt["runtime_h"], marker="*", linestyle="None", label="ADP")
    if sddp_rt["N"].size:
        plt.plot(sddp_rt["N"], sddp_rt["runtime_h"], marker="+", linestyle="None", label="SDDP")
    plt.xlabel("Number of samples")
    plt.ylabel("Running time (h)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out, bbox_inches="tight")
    plt.close()


def plot_std_last5(by_N: Dict[int, pd.DataFrame], out: Path):
    plt.figure(figsize=(6.0, 4.5), dpi=150)
    for n in sorted(by_N.keys()):
        df = by_N[n].sort_values("time")
        plt.plot(df["time"], df["value"], label=f"{n} samples")
    plt.xlabel("Time (h)")
    plt.ylabel("Standard deviation of last five samples ($)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out, bbox_inches="tight")
    plt.close()


# --- replace the whole function ---
def plot_first_hour_curve(adp_curve, sddp_curve, out: Path):
    """
    adp_curve / sddp_curve are (x, y, xname) or None.
    Chooses the x-label based on xname: if it's 'path' we say 'Scenario index'.
    """
    if adp_curve is None and sddp_curve is None:
        return

    def label_for_x(xname: str) -> str:
        if xname is None:
            return "X"
        xname_l = xname.strip().lower()
        if xname_l == "path":
            return "Scenario index at t=1"
        # looks like a level/state name
        return "Post-decision level at t=1"

    # pick x label from whichever we have
    xl = "X"
    for c in (adp_curve, sddp_curve):
        if c is not None:
            xl = label_for_x(c[2]); break

    plt.figure(figsize=(6.0, 4.0), dpi=150)
    if adp_curve is not None:
        x, y, _ = adp_curve
        idx = np.argsort(x)
        plt.plot(np.array(x)[idx], np.array(y)[idx], label="ADP")
    if sddp_curve is not None:
        x, y, _ = sddp_curve
        idx = np.argsort(x)
        plt.plot(np.array(x)[idx], np.array(y)[idx], label="SDDP")

    plt.xlabel(xl)
    plt.ylabel("Value function at t=1 ($)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(out, bbox_inches="tight")
    plt.close()

def find_first_hour_value_col(df: pd.DataFrame) -> str:
    for c in ["vbar2_at_xpost1","value","Vhat_hour1","v_est","vhat","v"]:
        if c in df.columns:
            return c
    # fallback: last numeric col
    nums = df.select_dtypes(include=["number"]).columns.tolist()
    return nums[-1] if nums else None

def load_first_hour_agg_by_N(root: Path, agg: str = "mean") -> Tuple[np.ndarray, np.ndarray]:
    """
    For each run_N*_first_hour.csv under `root`, compute an aggregate (mean/median)
    of the first-hour value column and return aligned arrays (N, value).
    """
    files = list(root.rglob("run_N*_first_hour.csv"))
    rows = []
    for f in files:
        m = re.search(r"run_N(\d+)_first_hour\.csv$", f.name, re.IGNORECASE)
        if not m: 
            continue
        N = int(m.group(1))
        try:
            df = pd.read_csv(f)
        except Exception:
            continue
        ycol = find_first_hour_value_col(df)
        if not ycol: 
            continue
        vals = pd.to_numeric(df[ycol], errors="coerce").dropna()
        if vals.empty:
            continue
        y = (vals.mean() if agg.lower()=="mean" else vals.median())
        rows.append((N, float(y)))
    if not rows:
        return np.array([]), np.array([])
    rows.sort(key=lambda t: t[0])
    Ns, Ys = zip(*rows)
    return np.array(Ns, dtype=int), np.array(Ys, dtype=float)

def plot_first_hour_vs_N(adp_xy: Tuple[np.ndarray, np.ndarray],
                         sddp_xy: Tuple[np.ndarray, np.ndarray],
                         out: Path,
                         agg_label: str = "mean"):
    """
    Plot first-hour value vs number of samples N with twin y-axes:
    - left y-axis: ADP
    - right y-axis: SDDP
    """
    fig, ax1 = plt.subplots(figsize=(6.0, 4.0), dpi=150)

    # Left axis for ADP
    Ns, Vy = adp_xy
    if Ns.size:
        ax1.plot(Ns, Vy, marker="*", color="tab:blue", label="ADP")
        ax1.set_ylabel(f"ADP first-hour value ({agg_label}) ($)", color="tab:blue")
        ax1.tick_params(axis="y", labelcolor="tab:blue")

    # Right axis for SDDP
    ax2 = ax1.twinx()
    Ns2, Vy2 = sddp_xy
    if Ns2.size:
        ax2.plot(Ns2, Vy2, marker="+", color="tab:orange", label="SDDP")
        ax2.set_ylabel(f"SDDP first-hour value ({agg_label}) ($)", color="tab:orange")
        ax2.tick_params(axis="y", labelcolor="tab:orange")

    ax1.set_xlabel("Number of samples")
    fig.tight_layout()
    fig.savefig(out, bbox_inches="tight")
    plt.close(fig)


# ---------------------------- Main ----------------------------

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--adp", default="adp_results")
    parser.add_argument("--sddp", default="SDDP_results")
    parser.add_argument("--eevws", default="results_eev_ws")
    parser.add_argument("--outdir", default="outputs")
    parser.add_argument("--debug", action="store_true")
    parser.add_argument(
    "--firsthour_agg",
    nargs="?",           # value is optional
    const="mean",        # if provided without a value, use 'mean'
    default="mean",
    choices=["mean","median"],
    help="Aggregation over scenarios for first-hour plot vs N"
)

    args, _ = parser.parse_known_args()

    work_root = Path(".work"); ensure_dir(work_root)
    outdir = Path(args.outdir); ensure_dir(outdir)

    def resolve(pstr): return unzip_if_needed(Path(pstr), work_root)

    adp_root, sddp_root, eev_root = resolve(args.adp), resolve(args.sddp), resolve(args.eevws)

    if args.debug:
        debug_inventory(adp_root,"ADP root")
        debug_inventory(sddp_root,"SDDP root")
        debug_inventory(eev_root,"EEV/WS root")

    # Runtime
    def load_runtime_generic(root: Path) -> Dict[str, np.ndarray]:
        rt = load_runtime_from_adp_runtime_by_N(root)
        if not rt["N"].size: rt = load_runtime_from_run_summary(root)
        return rt

    adp_rt = load_runtime_generic(adp_root)
    sddp_rt = load_runtime_generic(sddp_root)
    plot_runtime(adp_rt, sddp_rt, outdir/"fig_runtime.pdf")
    
    # --- First-hour value vs N (ADP vs SDDP) ---
    adp_N, adp_V = load_first_hour_agg_by_N(adp_root, agg=args.firsthour_agg)
    sddp_N, sddp_V = load_first_hour_agg_by_N(sddp_root, agg=args.firsthour_agg)
    plot_first_hour_vs_N((adp_N, adp_V), (sddp_N, sddp_V),
                        outdir / "fig_first_hour_vs_N.pdf",
                        agg_label=args.firsthour_agg)
    print(f"[ok] fig_first_hour_vs_N.pdf — saved to {outdir}")


    # Std curves
    by_N = load_std_from_perN_files(adp_root)
    if by_N:
        plot_std_last5(by_N, outdir/"fig_std_last5.pdf")

    # First hour
    adp_curve = load_first_hour_auto(adp_root)
    sddp_curve = load_first_hour_auto(sddp_root)
    plot_first_hour_curve(adp_curve, sddp_curve, outdir/"fig_first_hour_value.pdf")

    # EEV/WS tables
    eevws = load_eev_ws_from_pair_files(eev_root)
    if eevws is not None:
        eevws.to_csv(outdir/"eev_ws_table.csv", index=False)
        write_eev_ws_table_tex(eevws, outdir/"table_eev_ws.tex")
        print(f"[ok] EEV/WS tables saved to {outdir}")
    else:
        print("[warn] Could not load EEV/WS results; skipped table.")

    # OOS distributions
    oos_long = load_oos_stack(eev_root)
    if oos_long is not None:
        plot_oos_distributions_by_N(oos_long, outdir)
        print("[ok] Out-of-sample distribution figures saved.")
    else:
        print("[warn] No out-of-sample files found; skipped distributions.")

    # Efficiency frontier
  # Efficiency frontier (quality vs runtime)
    oos_long = load_oos_stack(eev_root)  # keep this
    # Build points per approach with the right runtime roots
    points = []
    for approach, rroot in [("ADP", adp_root), ("SDDP", sddp_root), ("EEV", eev_root), ("WS", eev_root)]:
        sub = oos_long[oos_long["approach"].str.lower() == approach.lower()] if oos_long is not None else None
        if sub is None or sub.empty:
            continue
        rt = load_runtime_from_adp_runtime_by_N(rroot)
        if not rt["N"].size:
            rt = load_runtime_from_run_summary(rroot)
        if not rt["N"].size:
            continue
        rt_map = {int(n): float(h) for n, h in zip(rt["N"], rt["runtime_h"])}
        for N, grp in sub.groupby("N"):
            q = float(pd.to_numeric(grp["value"], errors="coerce").mean())
            rh = rt_map.get(int(N))
            if rh is None:
                continue
            points.append({"approach": approach, "N": int(N), "runtime_h": rh, "quality": q})

    if points:
        eff_points = pd.DataFrame(points)
        plot_efficiency_frontier(eff_points, outdir/"fig_efficiency_frontier.pdf")
        print(f"[ok] fig_efficiency_frontier.pdf — saved to {outdir}")
    else:
        print("[warn] Could not build efficiency frontier (missing runtime or OOS).")

    # Policy t=1 (dispatch vs level)
    adp_pair = load_policy_t1_pair(adp_root)
    sddp_pair = load_policy_t1_pair(sddp_root)
    if adp_pair or sddp_pair:
        plot_policy_t1(adp_pair, sddp_pair, outdir/"fig_policy_t1_pi_vs_level.pdf")
        print(f"[ok] fig_policy_t1_pi_vs_level.pdf — saved to {outdir}")
    else:
        print("[warn] Could not find policy/level pairs for t=1; skipped policy plot.")

    print("Done.")
    
    


if __name__=="__main__":
    sys.argv = [sys.argv[0]] + [
        a for a in sys.argv[1:]
        if not (a.startswith("-f=") or a.startswith("--f="))
    ]
    main()


[ok] fig_first_hour_vs_N.pdf — saved to outputs
[ok] EEV/WS tables saved to outputs
[ok] Out-of-sample distribution figures saved.
[warn] Could not build efficiency frontier (missing runtime or OOS).
[warn] Could not find policy/level pairs for t=1; skipped policy plot.
Done.
